# Symmetries & Representations: Group Theory for Quantum Mechanics

**SIIEA Quantum Engineering Curriculum**
- **Curriculum Days:** Days 281-308
- **License:** CC BY-NC-SA 4.0 | Siiea Innovations, LLC

---

In [ ]:
# Hardware detection — adapts simulations to your machine
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("__file__")), ".."))
try:
    from hardware_config import HARDWARE, get_max_qubits
    print(f"Hardware: {HARDWARE['chip']} | {HARDWARE['memory_gb']} GB | Profile: {HARDWARE['profile']}")
    print(f"Max qubits: {get_max_qubits('safe')} (safe) / {get_max_qubits('max')} (max)")
except ImportError:
    print("hardware_config.py not found — using defaults")
    print("Run setup.sh from the repo root to generate it")

In [ ]:
# ── Imports & configuration ─────────────────────────────────────────
import numpy as np
from scipy.linalg import expm, block_diag
from itertools import permutations
import matplotlib.pyplot as plt
from matplotlib.patches import RegularPolygon, FancyArrowPatch
from mpl_toolkits.mplot3d import Axes3D
import sympy
from sympy import Matrix, eye, sqrt, Rational, pi, cos, sin, symbols
from sympy import pprint

%matplotlib inline

plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 15,
    'legend.fontsize': 11,
    'lines.linewidth': 2,
    'figure.dpi': 100,
})

print("All imports successful — ready for group theory.")

## 1. Permutation Groups: The Symmetric Group $S_3$

A **group** $(G, \cdot)$ satisfies four axioms:
1. **Closure:** $a, b \in G \Rightarrow a \cdot b \in G$
2. **Associativity:** $(a \cdot b) \cdot c = a \cdot (b \cdot c)$
3. **Identity:** $\exists\, e \in G$ such that $e \cdot a = a \cdot e = a$
4. **Inverse:** $\forall\, a \in G$, $\exists\, a^{-1}$ such that $a \cdot a^{-1} = e$

The **symmetric group** $S_n$ consists of all permutations of $n$ objects.
$S_3$ has $3! = 6$ elements and is the simplest non-abelian group.

### Elements of $S_3$

| Symbol | Cycle notation | Meaning |
|--------|---------------|---------|
| $e$ | $(1)(2)(3)$ | Identity |
| $r$ | $(123)$ | Rotation by $120°$ |
| $r^2$ | $(132)$ | Rotation by $240°$ |
| $s_1$ | $(23)$ | Reflection fixing vertex 1 |
| $s_2$ | $(13)$ | Reflection fixing vertex 2 |
| $s_3$ | $(12)$ | Reflection fixing vertex 3 |

### Quantum Connection
Identical particles in quantum mechanics are described by representations of the
symmetric group.  Bosons transform under the trivial representation (symmetric
wavefunctions), while fermions transform under the sign representation
(antisymmetric wavefunctions).

In [ ]:
# ── S₃ multiplication table ─────────────────────────────────────────
# Represent permutations as tuples: (a,b,c) means 1→a, 2→b, 3→c

def compose(p, q):
    """Compose two permutations: (p ∘ q)(i) = p(q(i))."""
    return tuple(p[q[i]-1] for i in range(len(p)))

# Define S₃ elements with names
S3_elements = {
    'e':   (1, 2, 3),   # identity
    'r':   (2, 3, 1),   # (123) rotation
    'r²':  (3, 1, 2),   # (132) rotation
    's₁':  (1, 3, 2),   # (23) reflection
    's₂':  (3, 2, 1),   # (13) reflection
    's₃':  (2, 1, 3),   # (12) reflection
}

names = list(S3_elements.keys())
perms = list(S3_elements.values())

# Build multiplication table
table = np.empty((6, 6), dtype=object)
for i, (ni, pi) in enumerate(zip(names, perms)):
    for j, (nj, pj) in enumerate(zip(names, perms)):
        product = compose(pi, pj)
        # Find which element this is
        for k, pk in enumerate(perms):
            if product == pk:
                table[i, j] = names[k]
                break

# Display as formatted table
print("S₃ Multiplication Table  (row ∘ column):")
print(f"{'':>4}", end="")
for n in names:
    print(f"{n:>5}", end="")
print()
print("-" * (4 + 5 * len(names)))
for i, n in enumerate(names):
    print(f"{n:>3} |", end="")
    for j in range(len(names)):
        print(f"{table[i,j]:>5}", end="")
    print()

# Verify non-commutativity
r, s1 = S3_elements['r'], S3_elements['s₁']
print(f"\nr ∘ s₁ = {compose(r, s1)} = s₃")
print(f"s₁ ∘ r = {compose(s1, r)} = s₂")
print(f"r ∘ s₁ ≠ s₁ ∘ r  →  S₃ is NON-ABELIAN ✓")

## 2. Symmetry Groups of Geometric Objects

The **dihedral group** $D_n$ is the symmetry group of a regular $n$-gon.
It has $2n$ elements: $n$ rotations and $n$ reflections.

- $D_3 \cong S_3$: symmetry group of an equilateral triangle (order 6)
- $D_4$: symmetry group of a square (order 8)

For $D_4$, the elements are:
- Rotations: $e, r, r^2, r^3$ (by $0°, 90°, 180°, 270°$)
- Reflections: $s_v, s_h, s_{d1}, s_{d2}$ (vertical, horizontal, two diagonals)

### Quantum Connection
Crystal symmetries form **space groups** that determine the band structure of
solids.  The irreducible representations of the crystal's point group label
the electronic energy bands and phonon modes.

In [ ]:
# ── Visualize D₃ and D₄ symmetries ─────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# D₃: equilateral triangle
ax = axes[0]
angles_tri = np.array([np.pi/2, np.pi/2 + 2*np.pi/3, np.pi/2 + 4*np.pi/3])
verts_tri = np.column_stack([np.cos(angles_tri), np.sin(angles_tri)])
triangle = plt.Polygon(verts_tri, fill=False, edgecolor='blue', linewidth=2.5)
ax.add_patch(triangle)

# Label vertices
for i, (x, y) in enumerate(verts_tri):
    ax.annotate(f'{i+1}', (x, y), fontsize=16, fontweight='bold',
                ha='center', va='center',
                xytext=(0.15*np.cos(angles_tri[i]), 0.15*np.sin(angles_tri[i])),
                textcoords='offset fontsize')

# Draw rotation axis and reflection axes
ax.plot(0, 0, 'ko', markersize=6)
for i in range(3):
    angle = np.pi/2 + i * 2*np.pi/3
    ax.plot([0, 1.3*np.cos(angle)], [0, 1.3*np.sin(angle)],
            'r--', alpha=0.5, linewidth=1.5)
    ax.annotate(f'$s_{i+1}$', (1.4*np.cos(angle), 1.4*np.sin(angle)),
                fontsize=13, color='red', ha='center')

ax.set_xlim(-1.8, 1.8)
ax.set_ylim(-1.5, 1.8)
ax.set_aspect('equal')
ax.set_title('$D_3$: Symmetries of the Triangle', fontsize=15)
ax.grid(True, alpha=0.2)

# D₄: square
ax = axes[1]
angles_sq = np.array([np.pi/4, 3*np.pi/4, 5*np.pi/4, 7*np.pi/4])
verts_sq = np.column_stack([np.cos(angles_sq), np.sin(angles_sq)])
square = plt.Polygon(verts_sq, fill=False, edgecolor='blue', linewidth=2.5)
ax.add_patch(square)

for i, (x, y) in enumerate(verts_sq):
    ax.annotate(f'{i+1}', (x, y), fontsize=16, fontweight='bold',
                ha='center', va='center',
                xytext=(0.15*np.cos(angles_sq[i]), 0.15*np.sin(angles_sq[i])),
                textcoords='offset fontsize')

# Reflection axes for D₄
ref_angles = [0, np.pi/2, np.pi/4, 3*np.pi/4]
ref_labels = ['$s_v$', '$s_h$', '$s_{d1}$', '$s_{d2}$']
for angle, label in zip(ref_angles, ref_labels):
    ax.plot([-1.3*np.cos(angle), 1.3*np.cos(angle)],
            [-1.3*np.sin(angle), 1.3*np.sin(angle)],
            'r--', alpha=0.5, linewidth=1.5)
    ax.annotate(label, (1.45*np.cos(angle), 1.45*np.sin(angle)),
                fontsize=13, color='red', ha='center')

ax.plot(0, 0, 'ko', markersize=6)
ax.set_xlim(-1.8, 1.8)
ax.set_ylim(-1.8, 1.8)
ax.set_aspect('equal')
ax.set_title('$D_4$: Symmetries of the Square', fontsize=15)
ax.grid(True, alpha=0.2)

plt.suptitle('Dihedral Group Symmetries', fontsize=17, y=1.02)
plt.tight_layout()
plt.show()

# Count elements
print(f"|D₃| = {2*3} = 2×3 (3 rotations + 3 reflections)")
print(f"|D₄| = {2*4} = 2×4 (4 rotations + 4 reflections)")

## 3. Representation Theory: Matrix Representations of Finite Groups

A **representation** of a group $G$ is a homomorphism $\rho: G \to GL(V)$
assigning to each group element an invertible matrix such that:

$$\rho(g_1 \cdot g_2) = \rho(g_1)\,\rho(g_2)$$

### Key definitions

- **Dimension** of a representation: the size of the matrices
- **Faithful** representation: $\rho$ is injective (different group elements get different matrices)
- **Irreducible representation (irrep):** has no nontrivial invariant subspaces
- **Reducible:** can be block-diagonalized into smaller representations

### Irreps of $S_3$

$S_3$ has exactly **3 irreducible representations**:

| Irrep | Dimension | Description |
|-------|-----------|-------------|
| **Trivial** ($\Gamma_1$) | 1 | All elements $\to 1$ |
| **Sign** ($\Gamma_2$) | 1 | Rotations $\to +1$, reflections $\to -1$ |
| **Standard** ($\Gamma_3$) | 2 | Faithful 2D representation |

Check: $1^2 + 1^2 + 2^2 = 6 = |S_3|$ ✓

### Quantum Connection
Irreducible representations label quantum states.  When the Hamiltonian commutes
with a symmetry group, energy eigenstates transform as irreps, giving rise to
quantum numbers and selection rules.

In [ ]:
# ── Matrix representations of S₃ ────────────────────────────────────
# Standard 2D representation (faithful): act on (x,y) plane

# 120° rotation matrix
theta = 2 * np.pi / 3
R120 = np.array([[np.cos(theta), -np.sin(theta)],
                  [np.sin(theta),  np.cos(theta)]])

# Reflection across x-axis
Sx = np.array([[1, 0], [0, -1]], dtype=float)

# Generate all 6 elements from r = R120 and s = Sx
I2 = np.eye(2)
reps_2d = {
    'e':   I2,
    'r':   R120,
    'r²':  R120 @ R120,
    's₁':  Sx,
    's₂':  R120 @ Sx,
    's₃':  R120 @ R120 @ Sx,
}

# Also build the trivial and sign representations
reps_trivial = {name: np.array([[1.0]]) for name in reps_2d}
reps_sign = {}
for name in reps_2d:
    if name in ['e', 'r', 'r²']:
        reps_sign[name] = np.array([[1.0]])
    else:
        reps_sign[name] = np.array([[-1.0]])

# Display the 2D representation
print("Standard 2D Representation of S₃:")
print("=" * 50)
for name, mat in reps_2d.items():
    print(f"\nρ({name}) =")
    for row in mat:
        print(f"  [{row[0]:+.4f}  {row[1]:+.4f}]")

# Verify homomorphism: ρ(r)·ρ(s₁) should equal ρ(s₃) = ρ(r∘s₁)
product = reps_2d['r'] @ reps_2d['s₁']
print(f"\nVerification: ρ(r) · ρ(s₁) =")
for row in product:
    print(f"  [{row[0]:+.4f}  {row[1]:+.4f}]")
print(f"\nρ(s₃) =")
for row in reps_2d['s₃']:
    print(f"  [{row[0]:+.4f}  {row[1]:+.4f}]")
diff = np.max(np.abs(product - reps_2d['s₃']))
print(f"\nMax difference: {diff:.2e} → Homomorphism verified ✓")

## 4. Character Tables & Orthogonality Relations

The **character** of a representation $\rho$ for group element $g$ is:

$$\chi_{\rho}(g) = \text{Tr}[\rho(g)]$$

Characters are constant on **conjugacy classes** (sets of elements related by
$g' = hgh^{-1}$).

### Great Orthogonality Theorem

For irreps $\alpha$ and $\beta$ of a finite group of order $|G|$:

$$\sum_{g \in G} \chi_\alpha(g)^* \chi_\beta(g) = |G|\,\delta_{\alpha\beta}$$

### Character Table of $S_3$

| | $\{e\}$ | $\{r, r^2\}$ | $\{s_1, s_2, s_3\}$ |
|------|---------|-------------|-------------------|
| $\Gamma_1$ (trivial) | 1 | 1 | 1 |
| $\Gamma_2$ (sign) | 1 | 1 | -1 |
| $\Gamma_3$ (standard) | 2 | -1 | 0 |

### Quantum Connection
Character tables predict which optical transitions are allowed.
A transition $|i\rangle \to |f\rangle$ under operator $\hat{O}$ is allowed only if
the product representation $\Gamma_i \otimes \Gamma_O \otimes \Gamma_f$ contains
the trivial representation.

In [ ]:
# ── Character table computation and orthogonality check ─────────────

# Conjugacy classes of S₃
classes = {
    '{e}': ['e'],
    '{r,r²}': ['r', 'r²'],
    '{s₁,s₂,s₃}': ['s₁', 's₂', 's₃'],
}

# Compute characters for each representation and class
all_reps = {
    'Γ₁ (trivial)': reps_trivial,
    'Γ₂ (sign)': reps_sign,
    'Γ₃ (standard)': reps_2d,
}

char_table = {}
print("Character Table of S₃")
print("=" * 55)
print(f"{'Irrep':<17} ", end="")
for cls_name in classes:
    print(f"{cls_name:>14}", end="")
print()
print("-" * 55)

for rep_name, rep_matrices in all_reps.items():
    chars = []
    for cls_name, cls_elements in classes.items():
        # Character = trace, same for all elements in a class
        chi = np.trace(rep_matrices[cls_elements[0]])
        chars.append(chi)
    char_table[rep_name] = chars
    print(f"{rep_name:<17} ", end="")
    for c in chars:
        print(f"{c:>14.1f}", end="")
    print()

# Verify Great Orthogonality Theorem
print("\n--- Orthogonality Check ---")
print("⟨χ_α | χ_β⟩ = (1/|G|) Σ |C_k| χ_α(C_k)* χ_β(C_k)")
class_sizes = [len(v) for v in classes.values()]
rep_names = list(char_table.keys())

for i, name_i in enumerate(rep_names):
    for j, name_j in enumerate(rep_names):
        inner = sum(
            class_sizes[k] * char_table[name_i][k] * char_table[name_j][k]
            for k in range(len(class_sizes))
        ) / 6.0  # |S₃| = 6
        status = "✓" if abs(inner - (1 if i == j else 0)) < 1e-10 else "✗"
        print(f"  ⟨{name_i}, {name_j}⟩ = {inner:.1f}  {status}")

## 5. Lie Groups: SO(2) and SO(3) Rotation Matrices

**Lie groups** are continuous groups that are also smooth manifolds.  They are
described by their **Lie algebra** — the tangent space at the identity.

### SO(2): Rotations in 2D

$$R(\theta) = \begin{pmatrix} \cos\theta & -\sin\theta \\ \sin\theta & \cos\theta \end{pmatrix}
= e^{i\theta J}, \quad J = \begin{pmatrix} 0 & -1 \\ 1 & 0 \end{pmatrix}$$

One parameter ($\theta$), one generator ($J$).

### SO(3): Rotations in 3D

Three generators (angular momentum operators):

$$J_x = \begin{pmatrix} 0&0&0 \\ 0&0&-1 \\ 0&1&0 \end{pmatrix}, \quad
J_y = \begin{pmatrix} 0&0&1 \\ 0&0&0 \\ -1&0&0 \end{pmatrix}, \quad
J_z = \begin{pmatrix} 0&-1&0 \\ 1&0&0 \\ 0&0&0 \end{pmatrix}$$

Lie algebra: $[J_i, J_j] = \epsilon_{ijk} J_k$

### Quantum Connection
Angular momentum in quantum mechanics comes from the representation theory of
SO(3).  The commutation relations $[\hat{L}_i, \hat{L}_j] = i\hbar\epsilon_{ijk}\hat{L}_k$
are exactly the SO(3) Lie algebra.

In [ ]:
# ── SO(2) and SO(3) rotations ───────────────────────────────────────
# SO(2): 2D rotations
def rotation_2d(theta):
    return np.array([[np.cos(theta), -np.sin(theta)],
                      [np.sin(theta),  np.cos(theta)]])

# Verify group properties
theta1, theta2 = np.pi/4, np.pi/3
R1 = rotation_2d(theta1)
R2 = rotation_2d(theta2)
R_product = R1 @ R2
R_sum = rotation_2d(theta1 + theta2)

print("SO(2) Verification:")
print(f"  R(π/4) · R(π/3) = R(π/4 + π/3)?")
print(f"  Max difference: {np.max(np.abs(R_product - R_sum)):.2e} ✓")
print(f"  det(R) = {np.linalg.det(R1):.6f} (should be 1)")
print(f"  R·Rᵀ = I? {np.allclose(R1 @ R1.T, np.eye(2))} ✓")

# SO(3): 3D rotation generators
Jx = np.array([[0, 0, 0], [0, 0, -1], [0, 1, 0]], dtype=float)
Jy = np.array([[0, 0, 1], [0, 0, 0], [-1, 0, 0]], dtype=float)
Jz = np.array([[0, -1, 0], [1, 0, 0], [0, 0, 0]], dtype=float)

# Verify Lie algebra: [Jx, Jy] = Jz
comm_xy = Jx @ Jy - Jy @ Jx
print(f"\nSO(3) Lie Algebra Verification:")
print(f"  [Jx, Jy] = Jz? {np.allclose(comm_xy, Jz)} ✓")
print(f"  [Jy, Jz] = Jx? {np.allclose(Jy @ Jz - Jz @ Jy, Jx)} ✓")
print(f"  [Jz, Jx] = Jy? {np.allclose(Jz @ Jx - Jx @ Jz, Jy)} ✓")

# Generate rotation matrices via matrix exponential
# R(θ, n̂) = exp(θ n̂·J)
theta = np.pi / 6
Rz = expm(theta * Jz)   # rotation about z by 30°
Rx = expm(theta * Jx)   # rotation about x by 30°

print(f"\nR_z(30°) via matrix exponential:")
print(Rz.round(6))
print(f"det = {np.linalg.det(Rz):.6f}, orthogonal: {np.allclose(Rz @ Rz.T, np.eye(3))}")

# Visualize: rotate a set of points
fig = plt.figure(figsize=(12, 5))

# Original and rotated vectors
v = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1],
              [1, 1, 0], [0, 1, 1], [1, 0, 1]], dtype=float)
v_rot = (expm(np.pi/4 * Jz) @ v.T).T

ax1 = fig.add_subplot(121, projection='3d')
for i, (orig, rot) in enumerate(zip(v, v_rot)):
    ax1.quiver(0, 0, 0, *orig, color=f'C{i}', alpha=0.5, linewidth=1.5,
               arrow_length_ratio=0.1)
    ax1.quiver(0, 0, 0, *rot, color=f'C{i}', linewidth=2.5,
               arrow_length_ratio=0.1)

ax1.set_xlabel('x'); ax1.set_ylabel('y'); ax1.set_zlabel('z')
ax1.set_title('SO(3) Rotation about z-axis (45°)')

# Demonstrate non-commutativity of 3D rotations
ax2 = fig.add_subplot(122, projection='3d')
angle = np.pi / 2

v0 = np.array([1, 0, 0])

# Path 1: Rx then Rz
v1_step1 = expm(angle * Jx) @ v0
v1_final = expm(angle * Jz) @ v1_step1

# Path 2: Rz then Rx
v2_step1 = expm(angle * Jz) @ v0
v2_final = expm(angle * Jx) @ v2_step1

ax2.quiver(0, 0, 0, *v0, color='black', linewidth=3, arrow_length_ratio=0.1,
           label='Original')
ax2.quiver(0, 0, 0, *v1_final, color='blue', linewidth=3, arrow_length_ratio=0.1,
           label='Rx then Rz')
ax2.quiver(0, 0, 0, *v2_final, color='red', linewidth=3, arrow_length_ratio=0.1,
           label='Rz then Rx')

ax2.set_xlabel('x'); ax2.set_ylabel('y'); ax2.set_zlabel('z')
ax2.set_title('Non-commutativity of 3D Rotations')
ax2.legend(loc='upper left')

plt.tight_layout()
plt.show()

print(f"\nRx·Rz applied to [1,0,0]: {v1_final.round(4)}")
print(f"Rz·Rx applied to [1,0,0]: {v2_final.round(4)}")
print(f"Different! 3D rotations do NOT commute (in general).")

## 6. SU(2) and Spin: Pauli Matrices as Generators

The group **SU(2)** consists of $2 \times 2$ unitary matrices with determinant 1.
It is the **double cover** of SO(3): every SO(3) rotation corresponds to two
SU(2) elements ($U$ and $-U$).

### Pauli Matrices (generators of SU(2))

$$\sigma_x = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}, \quad
\sigma_y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}, \quad
\sigma_z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}$$

### Key Properties

- $\sigma_i^2 = I$ for all $i$
- $\sigma_i \sigma_j = i\epsilon_{ijk}\sigma_k$ for $i \neq j$
- $\{\sigma_i, \sigma_j\} = 2\delta_{ij}I$
- $\text{Tr}(\sigma_i) = 0$, $\det(\sigma_i) = -1$

### SU(2) rotation

$$U(\hat{n}, \theta) = \exp\left(-i\frac{\theta}{2}\hat{n}\cdot\vec{\sigma}\right)
= \cos\frac{\theta}{2}\,I - i\sin\frac{\theta}{2}\,(\hat{n}\cdot\vec{\sigma})$$

### Quantum Connection
Spin-1/2 particles (electrons, protons, neutrons) are described by SU(2).
The spin operators are $\hat{S}_i = \frac{\hbar}{2}\sigma_i$.
SU(2) also underlies the electroweak force in the Standard Model.

In [ ]:
# ── Pauli matrices and SU(2) ────────────────────────────────────────
# Define Pauli matrices
sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)
I2 = np.eye(2, dtype=complex)

paulis = {'σx': sigma_x, 'σy': sigma_y, 'σz': sigma_z}

# Verify algebraic properties
print("=== Pauli Matrix Properties ===")
for name, sigma in paulis.items():
    print(f"\n{name}:")
    print(f"  σ² = I? {np.allclose(sigma @ sigma, I2)}")
    print(f"  Tr(σ) = {np.trace(sigma):.1f}")
    print(f"  det(σ) = {np.linalg.det(sigma):.1f}")
    print(f"  Hermitian? {np.allclose(sigma, sigma.conj().T)}")
    print(f"  Unitary? {np.allclose(sigma @ sigma.conj().T, I2)}")

# Commutation relations: [σi, σj] = 2i εijk σk
print("\n=== Commutation Relations ===")
comm_xy = sigma_x @ sigma_y - sigma_y @ sigma_x
print(f"[σx, σy] = 2i·σz? {np.allclose(comm_xy, 2j * sigma_z)} ✓")
comm_yz = sigma_y @ sigma_z - sigma_z @ sigma_y
print(f"[σy, σz] = 2i·σx? {np.allclose(comm_yz, 2j * sigma_x)} ✓")
comm_zx = sigma_z @ sigma_x - sigma_x @ sigma_z
print(f"[σz, σx] = 2i·σy? {np.allclose(comm_zx, 2j * sigma_y)} ✓")

# Anti-commutation: {σi, σj} = 2δij I
print("\n=== Anti-commutation Relations ===")
acomm_xy = sigma_x @ sigma_y + sigma_y @ sigma_x
print(f"{{σx, σy}} = 0? {np.allclose(acomm_xy, 0)} ✓")
acomm_xx = sigma_x @ sigma_x + sigma_x @ sigma_x
print(f"{{σx, σx}} = 2I? {np.allclose(acomm_xx, 2*I2)} ✓")

# SU(2) rotation: spin-1/2 state rotated about z-axis
print("\n=== SU(2) Rotation of Spin-1/2 ===")
theta_values = [0, np.pi/2, np.pi, 2*np.pi, 4*np.pi]
spin_up = np.array([1, 0], dtype=complex)

for theta in theta_values:
    U = expm(-1j * theta/2 * sigma_z)
    rotated = U @ spin_up
    print(f"  θ = {theta/np.pi:.1f}π: U|↑⟩ = [{rotated[0]:.4f}, {rotated[1]:.4f}]"
          f"  (phase: e^{{-iθ/2}} = {np.exp(-1j*theta/2):.4f})")

print("\nNote: 2π rotation gives -|↑⟩ (spinor sign flip!)")
print("      4π rotation restores |↑⟩ — this is the SU(2) double cover of SO(3)")

## 7. Clebsch-Gordan Coefficients & Angular Momentum Addition

When two angular momenta $j_1$ and $j_2$ combine, the total angular momentum
$J$ ranges from $|j_1 - j_2|$ to $j_1 + j_2$:

$$|j_1, m_1\rangle \otimes |j_2, m_2\rangle = \sum_{J, M} \langle j_1, m_1; j_2, m_2 | J, M \rangle \, |J, M\rangle$$

The coefficients $\langle j_1, m_1; j_2, m_2 | J, M \rangle$ are the
**Clebsch-Gordan coefficients**.

### Selection Rules
- $M = m_1 + m_2$
- $|j_1 - j_2| \leq J \leq j_1 + j_2$
- Triangle inequality: $j_1 + j_2 \geq J \geq |j_1 - j_2|$

### Common case: two spin-1/2 particles

$$\frac{1}{2} \otimes \frac{1}{2} = 1 \oplus 0$$

The triplet ($J=1$) and singlet ($J=0$) states:

$$|1,1\rangle = |{\uparrow\uparrow}\rangle, \quad
|1,0\rangle = \frac{1}{\sqrt{2}}(|{\uparrow\downarrow}\rangle + |{\downarrow\uparrow}\rangle), \quad
|1,-1\rangle = |{\downarrow\downarrow}\rangle$$

$$|0,0\rangle = \frac{1}{\sqrt{2}}(|{\uparrow\downarrow}\rangle - |{\downarrow\uparrow}\rangle)$$

### Quantum Connection
Clebsch-Gordan coefficients are essential for adding angular momenta in
multi-electron atoms, nuclear physics, and particle physics.  They determine
the spectral fine structure of hydrogen.

In [ ]:
# ── Clebsch-Gordan coefficients for two spin-1/2 particles ─────────
from sympy.physics.quantum.cg import CG
from sympy import S

print("=== Clebsch-Gordan Coefficients: ½ ⊗ ½ → 1 ⊕ 0 ===\n")

j1, j2 = S(1)/2, S(1)/2

# Enumerate all possible |j1,m1⟩|j2,m2⟩ → |J,M⟩
print(f"{'|j1,m1; j2,m2⟩':<25} {'|J,M⟩':<12} {'CG coeff':>12}")
print("-" * 50)

for m1 in [S(1)/2, -S(1)/2]:
    for m2 in [S(1)/2, -S(1)/2]:
        M = m1 + m2
        for J in [S(1), S(0)]:
            if abs(M) <= J:
                cg = CG(j1, m1, j2, m2, J, M).doit()
                if cg != 0:
                    m1_str = "↑" if m1 > 0 else "↓"
                    m2_str = "↑" if m2 > 0 else "↓"
                    print(f"|{m1_str}{m2_str}⟩ = |½,{float(m1):+.1f}; ½,{float(m2):+.1f}⟩"
                          f"  |{float(J):.0f},{float(M):+.0f}⟩"
                          f"  {float(cg):+.4f}")

# Build the coupled states explicitly
print("\n=== Coupled States ===")
print("Triplet (J=1, symmetric):")
print("  |1,+1⟩ = |↑↑⟩")
print("  |1, 0⟩ = (1/√2)(|↑↓⟩ + |↓↑⟩)")
print("  |1,-1⟩ = |↓↓⟩")
print("\nSinglet (J=0, antisymmetric):")
print("  |0, 0⟩ = (1/√2)(|↑↓⟩ - |↓↑⟩)")

# Verify with tensor product of Pauli matrices
# S_total = S_1 ⊗ I + I ⊗ S_2
# S² = S_total · S_total
Sx_tot = np.kron(sigma_x/2, I2) + np.kron(I2, sigma_x/2)
Sy_tot = np.kron(sigma_y/2, I2) + np.kron(I2, sigma_y/2)
Sz_tot = np.kron(sigma_z/2, I2) + np.kron(I2, sigma_z/2)
S2_tot = Sx_tot @ Sx_tot + Sy_tot @ Sy_tot + Sz_tot @ Sz_tot

print("\n=== Verification: S² eigenvalues ===")
evals = np.linalg.eigvalsh(S2_tot.real)
print(f"S² eigenvalues: {sorted(evals)}")
print(f"Expected: J(J+1) = 0·1=0 (singlet), 1·2=2,2,2 (triplet)")
print(f"Match: {np.allclose(sorted(evals), [0, 2, 2, 2])} ✓")

In [ ]:
# ── Higher angular momentum: j=1 ⊗ j=1/2 ──────────────────────────
# 1 ⊗ 1/2 = 3/2 ⊕ 1/2

j1_val, j2_val = S(1), S(1)/2

print("=== Clebsch-Gordan Table: j=1 ⊗ j=½ → j=3/2 ⊕ j=½ ===\n")

# Build full CG table
m1_vals = [S(1), S(0), -S(1)]
m2_vals = [S(1)/2, -S(1)/2]
J_vals = [S(3)/2, S(1)/2]

print(f"{'m1':>5} {'m2':>5} {'M':>5} {'J=3/2':>10} {'J=1/2':>10}")
print("-" * 40)

for m1 in m1_vals:
    for m2 in m2_vals:
        M = m1 + m2
        row = f"{float(m1):>+5.1f} {float(m2):>+5.1f} {float(M):>+5.1f}"
        for J in J_vals:
            if abs(M) <= J:
                cg = float(CG(j1_val, m1, j2_val, m2, J, M).doit())
                row += f" {cg:>+10.4f}"
            else:
                row += f" {'---':>10}"
        print(row)

# Dimension check
dim_product = int(2*j1_val + 1) * int(2*j2_val + 1)
dim_sum = sum(int(2*J + 1) for J in J_vals)
print(f"\nDimension check: {int(2*j1_val+1)} × {int(2*j2_val+1)} = {dim_product}")
print(f"                 {int(2*J_vals[0]+1)} + {int(2*J_vals[1]+1)} = {dim_sum}")
print(f"                 {dim_product} = {dim_sum} ✓")

## Summary: Group Theory for Quantum Mechanics

| Concept | Mathematical Object | Quantum Application |
|---------|-------------------|---------------------|
| Permutations ($S_n$) | Finite group | Identical particles (bosons/fermions) |
| Dihedral ($D_n$) | Finite group | Molecular symmetries, crystal field theory |
| Representations | Group $\to$ matrices | Quantum numbers, selection rules |
| Character tables | Traces of irreps | Symmetry-allowed transitions |
| SO(3) | Lie group | Orbital angular momentum $L$ |
| SU(2) | Lie group (double cover) | Spin angular momentum $S$ |
| Clebsch-Gordan | Coupling coefficients | Angular momentum addition |

### The Deep Connection

**Noether's theorem** links every continuous symmetry to a conservation law:

| Symmetry | Conservation Law | Generator |
|----------|-----------------|-----------|
| Translation in time | Energy | $\hat{H}$ |
| Translation in space | Momentum | $\hat{p}$ |
| Rotation | Angular momentum | $\hat{L}$ |
| Gauge (U(1)) | Electric charge | $\hat{Q}$ |

Group theory is not merely a mathematical tool — it is the **language** in which
the fundamental laws of quantum physics are written.

---
*Notebook generated for the SIIEA Quantum Engineering curriculum.*
*License: CC BY-NC-SA 4.0 | Siiea Innovations, LLC*